This notebook is used to check the migration of PBs method from SimaPro to Brightway2

For that, its performance will be checked against the dataset of chemicals from Green Chemistry paper.

# Read set of SimaPro solved chemicals

In [1]:
import re
from pathlib import Path

import brightway2 as bw
import pandas as pd
from project_path import ROOT_DIR
import project_path
from src.utils import progressbar

In [2]:
import bw2data

In [3]:
from bw_recipe_2016 import add_recipe_2016
import aesa_pbs
aesa_pbs.__version__

'0.8.1'

In [4]:
SIMAPRO_DATA = Path(ROOT_DIR + r"\data\raw\lcia-results-from-sp910-combined.xlsx")

In [5]:
with pd.ExcelFile(SIMAPRO_DATA, engine=None) as xl_obj:
    df_simapro = pd.read_excel(xl_obj, sheet_name="Sheet1")

In [6]:
df_simapro.head(3)

,wkbName,Activity,"('PBs-LCIA V0.71 V0.71', 'Climate change - CO2 concentration', 'ppm')","('PBs-LCIA V0.71 V0.71', 'Climate change - Energy imbalance', 'Wm-2')","('PBs-LCIA V0.71 V0.71', 'Stratospheric ozone depletion', 'DU')","('PBs-LCIA V0.71 V0.71', 'Ocean acidification', 'Omega Aragon')","('PBs-LCIA V0.71 V0.71', 'Biogeochemical flows - P', 'Tg P')","('PBs-LCIA V0.71 V0.71', 'Biogeochemical flows - N', 'Tg N')","('PBs-LCIA V0.71 V0.71', 'Land-system change - Global', '%')","('PBs-LCIA V0.71 V0.71', 'Freshwater use - Global', 'km3')",...,"('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Human health', 'DALY')","('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Ecosystems', 'species.yr')","('ReCiPe 2016 Endpoint (H) V1.03 / World (2010) H/A', 'Resources', 'USD2013')","('IPCC 2013 GWP 100a V1.03', 'IPCC GWP 100a', 'kg CO2 eq')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non renewable, fossil', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non-renewable, nuclear', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Non-renewable, biomass', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, biomass', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, wind, solar, geothe', 'MJ')","('Cumulative Energy Demand V1.11 / Cumulative energy demand', 'Renewable, water', 'MJ')"
0,Chemicals-Acids (inorganic)-Market,Sulfuric acid {RoW}| market for sulfuric acid ...,3.682300e-12,4.965498e-14,1.156094e-15,1.125213e-14,2.047673e-16,3.232508e-14,1.099348e-16,7.588386e-13,...,0.000002,2.389076e-09,0.052662,0.156735,6.316318,0.151773,0.000150,0.121871,0.011526,0.087806
1,Chemicals-Acids (inorganic)-Market,Sulfuric acid {RER}| market for sulfuric acid ...,2.682016e-12,3.621578e-14,8.425343e-16,8.195512e-15,2.162822e-16,3.982650e-14,1.233763e-16,2.679813e-13,...,0.000002,2.382083e-09,0.060525,0.119277,7.175078,0.233407,0.000138,0.073102,0.016586,0.064712
2,Chemicals-Acids (inorganic)-Market,Sulfamic acid {GLO}| market for sulfamic acid ...,5.439020e-11,7.271744e-13,8.913290e-15,1.662025e-13,1.122301e-15,2.008279e-13,5.993186e-16,4.956729e-12,...,0.000007,1.325358e-08,0.272812,2.172702,35.989036,2.025470,0.000812,0.479571,0.171391,0.812977


In [7]:
# df_simapro.columns

In [8]:
[i for i in df_simapro.columns if "ReCiPe 2016 Midpoint" in i]

["('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Stratospheric ozone depletion', 'kg CFC11 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ionizing radiation', 'kBq Co-60 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Human health', 'kg NOx eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fine particulate matter formation', 'kg PM2.5 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Terrestrial ecosystems', 'kg NOx eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial acidification', 'kg SO2 eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater eutrophication', 'kg P eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine eutrophication', 'kg N eq')",
 "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial ecotoxicity', 'kg 1,4-DCB')",
 "('ReCiPe

In [9]:
[i for i in df_simapro.columns if "0.72" in i]

["('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')",
 "('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')",
 "('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')",
 "('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')",
 "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')",
 "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')",
 "('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')",
 "('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')",
 "('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')"]

```python
# all ReCiPe2016 categories implemented in bw_recipe_2016
# use them to update `map_methods`
[
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Mineral Resource Scarcity", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Fossil resource scarcity"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ozone Formation", "Damage to Ecosystems"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Marine eutrophication"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Terrestrial Acidification"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ionizing Radiation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ozone Formation", "Damage to Humans"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Stratospheric Ozone Depletion", "Infinite timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Marine", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Terrestrial", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Land occupation"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Freshwater Eutrophication"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Ecotoxicity", "Freshwater", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Water consumption"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Hierarchist", ),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Particulate Matter Formation", "Egalitarian", ),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Land transformation"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Midpoint", "Toxicity", "Non-carcinogenic", "Egalitarian"),


("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Stratospheric Ozone Depletion", "Infinite timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ionizing Radiation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Particulate Matter Formation", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Ozone Formation", "Damage to Humans", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Toxicity", "Non-carcinogenic", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Water consumption", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "100 year timescale", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Global Warming", "1000 year timescale", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ozone Formation", "Damage to Ecosystems", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Terrestrial Acidification", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Ecotoxicity", "Terrestrial", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Water consumption", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land occupation", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "20 year timescale", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "100 year timescale","Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Global Warming", "1000 year timescale","Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Freshwater Eutrophication", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Ecotoxicity", "Freshwater", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Water consumption", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Ecotoxicity", "Marine","Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Marine eutrophication", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Mineral Resource Scarcity", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Land transformation", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Individualist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Fossil resource scarcity", "Egalitarian"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Freshwater ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Resources", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Human health", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Marine ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Terrestrial ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Ecosystems", "Aggregated", "Individualist"),

("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Egalitarian"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Hierarchist"),
("ReCiPe 2016", "1.1 (20180117)", "Endpoint", "Weighted single score", "Aggregated", "Individualist"),
]
```

In [84]:
map_methods = {
    "Activity": "Activity",
    "('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')": ('AESA (PBs-LCIA)', 'freshwater use', 'global'),
    "('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')": ('AESA (PBs-LCIA)', 'climate change', 'atmospheric CO2 concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')": ('AESA (PBs-LCIA)', 'change in biosphere integrity', 'functional diversity'),
    "('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')": ('AESA (PBs-LCIA)', 'ocean acidification', 'carbonate ion concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')": ('AESA (PBs-LCIA)', 'biogeochemical flows', 'phosphorus'),
    "('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')": ('AESA (PBs-LCIA)', 'climate change','energy imbalance at top-of-atmosphere'),
    "('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')": ('AESA (PBs-LCIA)', 'biogeochemical flows', 'nitrogen'),
    "('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')": ('AESA (PBs-LCIA)', 'ozone depletion', 'stratospheric O3 concentration'),
    "('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')": ('AESA (PBs-LCIA)', 'land-system change', 'global'),
    "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq')": ('ReCiPe 2016', '1.1 (20180117)', 'Midpoint', 'Global Warming', '100 year timescale', 'Hierarchist'),
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Stratospheric ozone depletion', 'kg CFC11 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ionizing radiation', 'kBq Co-60 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Human health', 'kg NOx eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fine particulate matter formation', 'kg PM2.5 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Ozone formation, Terrestrial ecosystems', 'kg NOx eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial acidification', 'kg SO2 eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater eutrophication', 'kg P eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine eutrophication', 'kg N eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Terrestrial ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Freshwater ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Marine ecotoxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Human carcinogenic toxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Human non-carcinogenic toxicity', 'kg 1,4-DCB')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Land use', 'm2a crop eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Mineral resource scarcity', 'kg Cu eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Fossil resource scarcity', 'kg oil eq')": ,
    # "('ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Water consumption', 'm3'): "
}

In [85]:
rev_map_methods = {v: k for k, v in map_methods.items()}

In [89]:
df_sp_selected = df_simapro.rename(columns=map_methods)[map_methods.values()]
df_sp_selected.head(3)

C:\Users\ViteksPC\VirtualEnvs\bw2\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


,Activity,"(AESA (PBs-LCIA), freshwater use, global)","(AESA (PBs-LCIA), climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), change in biosphere integrity, functional diversity)","(AESA (PBs-LCIA), ocean acidification, carbonate ion concentration)","(AESA (PBs-LCIA), biogeochemical flows, phosphorus)","(AESA (PBs-LCIA), climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), biogeochemical flows, nitrogen)","(AESA (PBs-LCIA), ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), land-system change, global)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)"
0,Sulfuric acid {RoW}| market for sulfuric acid ...,7.58839e-13,3.68707e-12,4.28060e-14,1.12667e-14,2.04767e-16,4.97175e-14,3.23251e-14,1.15609e-15,1.09935e-16,1.60737e-01
1,Sulfuric acid {RER}| market for sulfuric acid ...,2.67981e-13,2.68528e-12,3.34008e-14,8.20548e-15,2.16282e-16,3.62586e-14,3.98265e-14,8.42534e-16,1.23376e-16,1.23240e-01
2,Sulfamic acid {GLO}| market for sulfamic acid ...,4.95673e-12,5.44321e-11,5.04637e-13,1.66331e-13,1.12230e-15,7.27725e-13,2.00828e-13,8.91329e-15,5.99319e-16,2.20239e+00


In [90]:
len(list(df_sp_selected.Activity))

947

In [91]:
# list(df_simapro.Activity)

In [92]:
pattern = re.compile(r"(^.*) {(.*)}\| (.*) \|")

In [93]:
activities_sp = {}
for i in list(df_sp_selected.Activity):
    mo = re.match(pattern, i)
    if mo:
        ref_prod = mo[1].lower()
        location = mo[2]
        name = mo[3].lower()
        activities_sp[i] = {"reference product": ref_prod, "name": name, "location": location}
# activities_sp

# Prepare bw project

In [94]:
bw.projects

Brightway2 projects manager with 14 objects:
	StepByStep
	bw2_class_2020_example_database
	bw2_class_2020_intro
	car comparison paper
	check_PBsmethod
	check_electricity_EES_paper
	check_impact_electricity
	default
	ecoinvent 3.6
	global_power_mix
	parameters - manual creation
	premise_scenarios
	simapro-ecoinvent-import
	trial-import_from_SimaPro
Use `projects.report()` to get a report on all projects.

In [95]:
bw.projects.set_current("check_PBsmethod")

In [96]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei35apos

In [97]:
bw.projects.dir

'C:\\Users\\ViteksPC\\AppData\\Local\\pylca\\Brightway3\\check_PBsmethod.fbe9669d91325ca0149ce34bf0d8ff48'

In [98]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


In [99]:
if "ei35apos" in bw.databases:
    print("Database already present! No import is needed.")
else:
    PATH_TO_EI_DATASETS = r"C:\Users\ViteksPC\Documents\00-ETH_projects\18-BW2Hub\01.Data\Ecoinvent\v3.5\ecoinvent 3.5_apos_ecoSpold02\datasets"
    ei = bw.SingleOutputEcospold2Importer(PATH_TO_EI_DATASETS, "ei35apos")
    # ei
    ei.apply_strategies()
    ei.statistics()

    # write database to disk --> actually saves it
    ei.write_database()
    del ei

Database already present! No import is needed.


In [100]:
bw.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei35apos

In [101]:
if any("AESA" in str(aesa) for aesa in bw.methods):
    print("AESA methods already present!")
else:
    aesa_pbs.add_aesa_pbs(verbose=False)

AESA methods already present!


In [102]:
if any(rcp for rcp in bw.methods if "recipe 2016" in str(rcp).lower()):
    print("ReCiPe2016 methods already present!")
else:
    add_recipe_2016()

ReCiPe2016 methods already present!


In [103]:
recipe2016 = [rcp for rcp in bw.methods if "recipe 2016" in str(rcp).lower()]
pbs = [aesa for aesa in bw.methods if "AESA" in str(aesa)]

In [109]:
# for m in recipe2016:
for m in pbs:
    for k, v in rev_map_methods.items():
        if str(k) in str(m):
            print(v)

('PBs-LCIA (baseline) V0.72', 'Climate change - CO2 concentration', 'ppm')
('PBs-LCIA (baseline) V0.72', 'Change in biosphere integrity - BII loss', '% BII loss')
('PBs-LCIA (baseline) V0.72', 'Stratospheric ozone depletion', 'DU')
('PBs-LCIA (baseline) V0.72', 'Freshwater use - Global', 'km3')
('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - P', 'Tg P')
('PBs-LCIA (baseline) V0.72', 'Biogeochemical flows - N', 'Tg N')
('PBs-LCIA (baseline) V0.72', 'Climate change - Energy imbalance', 'Wm-2')
('PBs-LCIA (baseline) V0.72', 'Land-system change - Global', '%')
('PBs-LCIA (baseline) V0.72', 'Ocean acidification', 'Omega Aragon')


In [110]:
eidb = bw.Database("ei35apos")
# eidb

In [111]:
trial = eidb.random()

In [112]:
trial.as_dict().keys()

dict_keys(['comment', 'classifications', 'activity type', 'activity', 'database', 'filename', 'location', 'name', 'synonyms', 'parameters', 'authors', 'type', 'reference product', 'flow', 'unit', 'production amount', 'code'])

In [113]:
trial["name"]

'sawnwood production, hardwood, raw, dried (u=20%)'

In [114]:
trial.as_dict()["name"]

'sawnwood production, hardwood, raw, dried (u=20%)'

In [115]:
trial.as_dict()["code"]

'7bb54b30bf31334eec48d9d7fa7abd62'

In [116]:
trial.key

('ei35apos', '7bb54b30bf31334eec48d9d7fa7abd62')

In [117]:
# [i for i in trial.biosphere()]

In [118]:
# [i for i in trial.exchanges()]

In [119]:
[i for i in trial.production()]

[Exchange: 1.0 cubic meter 'sawnwood production, hardwood, raw, dried (u=20%)' (cubic meter, RoW, None) to 'sawnwood production, hardwood, raw, dried (u=20%)' (cubic meter, RoW, None)>]

In [120]:
# [i for i in trial.substitution()]
# [i for i in trial.technosphere()]

In [121]:
# [i for i in trial.upstream()]

In [122]:
# Moved to src, try to import it

# import sys
# from datetime import datetime

# def progressbar(itobj, **kwargs):
#     """Generate a progress bar.

#     Progress bar adapted from https://stackoverflow.com/a/34482761/14485040

#     Parameters
#     ----------
#     itobj : list-like
#         Iterable object

#     Returns
#     -------
#     Print out a progress bar

#     Example
#     -------
#     lst = [5, 3, 4]
#     for i in progressbar(itobj=lst, total=len(lst), prefix="Progress: ", size=5, unit="datapoint"):
#         pass
#         # do_something

#     **kwargs
#     --------
#     total : int, default len(itobj)
#         Size of the iterable object
#     prefix : str, default ""
#         String preceding the progress bar
#     size : int, default 30
#         Number of icons conforming the progress bar
#     unit : str, default ""
#         String following the progress bar, intended to be used for units
#     icon_done : str, default "🟩"
#         str-like symbol to be used as "done"-icon
#     icon_todo : str, default "🔳"
#         str-like symbol to be used as "to do"-icon
#     """
#     start_time = datetime.now()
#     file = sys.stdout

#     try:
#         iter(itobj)  # check if provided `itobj` is iterable
#     except TypeError as err:
#         raise TypeError from err

#     # if "total" in kwargs:
#     #     total = kwargs["total"]
#     # else:
#     #     total = len(itobj)
#     # refactor the above to:
#     try:
#         len_itobj = len(itobj)
#     except TypeError:
#         len_itobj = int(5)  # placeholder
#     total = kwargs.get("total", len_itobj)
#     # similarly for the rest
#     prefix = kwargs.get("prefix", "")
#     size = kwargs.get("size", 30)
#     unit = kwargs.get("unit", "")
#     icon_done = kwargs.get("icon_done", "🟩")
#     icon_todo = kwargs.get("icon_todo", "🔳")

#     def show(j):
#         icons_to_show = int(size * j / total)
#         # some ASCII symbol or emoji alternatives

#         # |, *, #, %, ▒. ▓, █, ■, ♢, ⚃ ⚄ ⚅
#         # 🦾, 👍, 🔴, 🟡 🟢 🔵 🟣 ⚫️ ⚪️, 🔸 🔹 🔶 🔷 🔳 🔲 ▪️ ▫️ ◾️ ◽️ ◼️ ◻️ 🟥 🟧 🟨 🟩 🟦 🟪 ⬛️ ⬜️, ▶️

#         file.write(
#             "%s[%s%s] %i/%i %s\r"
#             % (
#                 prefix,
#                 icon_done * icons_to_show,
#                 icon_todo * (size - icons_to_show),
#                 j,
#                 total,
#                 unit,
#             )
#         )
#         file.flush()

#     show(0)
#     for i, item in enumerate(itobj):
#         yield item
#         show(i + 1)
#     file.write("\n")
#     file.flush()

#     end_time = datetime.now()
#     print('Duration: {}'.format(end_time - start_time))

In [123]:
activities_sp['Sulfuric acid {RoW}| market for sulfuric acid | APOS, S'].keys()

dict_keys(['reference product', 'name', 'location'])

In [124]:
eidb_markets = [activity for activity in eidb if "market" in activity["name"]]

activities_bw = []

for act_sp in progressbar(
    itobj=activities_sp.keys(),
    prefix="Progress: ",
    size=20,
    unit="activity(ies)",
):
    # activities_sp.setdefault("spname", None)
    for act in eidb_markets:
        if (
            activities_sp[act_sp]["reference product"].lower() == str(act["reference product"]).lower()
            and "market" in str(act["name"])
            and activities_sp[act_sp]["location"] == str(act["location"])
        ):
            activities_bw.append(act)
            activities_sp[act_sp].update({"einame": act})


Progress: [🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩] 947/947 activity(ies)
Duration: 0:00:06.947095


In [125]:
[i for i, a in activities_sp.items() if "market" in a.get("einame", "nope")]

[]

In [126]:
# len(
#     set(activities_bw)- set([b["spname"] for b in activities_sp.values() if b.get("spname")])
# )

# activities not returned to activities_sp!!!
# STOPPED HERE!
set(activities_bw)- set([b["einame"] for b in activities_sp.values() if b.get("einame")])

{'market for neutralising agent, sodium hydroxide-equivalent' (kilogram, GLO, None),
 'fraction 1 from naphtha separation to generic market for chemical, organic' (kilogram, GLO, None),
 'fraction 8 from naphtha separation to generic market for chemical, organic' (kilogram, GLO, None),
 'methylcyclopentane to generic market for solvent, organic' (kilogram, GLO, None),
 '1,1-dimethylcyclopentane to generic market for solvent, organic' (kilogram, GLO, None),
 'white spirit to generic market for solvent for paint' (kilogram, GLO, None),
 'praseodymium oxide to generic market for mischmetal' (kilogram, GLO, None),
 'soda ash, dense, to generic market for neutralising agent' (kilogram, GLO, None),
 'acetonitrile to generic market for solvent for chromatography analysis' (kilogram, GLO, None),
 'tin dioxide to generic market for polishing powder' (kilogram, GLO, None),
 'market for vegetable oil, refined' (kilogram, GLO, None),
 'market for soil pH raising agent, as CaCO3' (kilogram, GLO, No

In [127]:
len(activities_bw)

978

In [128]:
978-33

945

In [129]:
len(set(activities_bw))

978

In [130]:
871+76

947

In [131]:
len(activities_sp)

947

In [132]:
[i for i in activities_sp if activities_sp[i].get("einame", None)==None]

['Deinking emulsion, in paper production {GLO}| market for | APOS, S',
 'Rare earth concentrate, 70% REO, from bastnasite {GLO}| market for | APOS, U - copy']

In [133]:
activities_sp['Sulfuric acid {RoW}| market for sulfuric acid | APOS, S']["einame"]

'market for sulfuric acid' (kilogram, RoW, None)

In [134]:
def get_einame(key):
    return activities_sp[key].get("einame", None)#["activity"]

In [135]:
get_einame('Sulfuric acid {RoW}| market for sulfuric acid | APOS, S')

'market for sulfuric acid' (kilogram, RoW, None)

In [136]:
df_sp_selected['einame'] = df_sp_selected['Activity'].apply(lambda key: get_einame(key))
df_sp_selected

,Activity,"(AESA (PBs-LCIA), freshwater use, global)","(AESA (PBs-LCIA), climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), change in biosphere integrity, functional diversity)","(AESA (PBs-LCIA), ocean acidification, carbonate ion concentration)","(AESA (PBs-LCIA), biogeochemical flows, phosphorus)","(AESA (PBs-LCIA), climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), biogeochemical flows, nitrogen)","(AESA (PBs-LCIA), ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), land-system change, global)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)",einame
0,Sulfuric acid {RoW}| market for sulfuric acid ...,7.58839e-13,3.68707e-12,4.28060e-14,1.12667e-14,2.04767e-16,4.97175e-14,3.23251e-14,1.15609e-15,1.09935e-16,1.60737e-01,"[comment, classifications, activity type, acti..."
1,Sulfuric acid {RER}| market for sulfuric acid ...,2.67981e-13,2.68528e-12,3.34008e-14,8.20548e-15,2.16282e-16,3.62586e-14,3.98265e-14,8.42534e-16,1.23376e-16,1.23240e-01,"[comment, classifications, activity type, acti..."
2,Sulfamic acid {GLO}| market for sulfamic acid ...,4.95673e-12,5.44321e-11,5.04637e-13,1.66331e-13,1.12230e-15,7.27725e-13,2.00828e-13,8.91329e-15,5.99319e-16,2.20239e+00,"[comment, classifications, activity type, acti..."
3,"Phosphoric acid, industrial grade, without wat...",2.07610e-11,3.68338e-11,7.17016e-13,1.12554e-13,5.74035e-14,4.92490e-13,4.25982e-13,6.92091e-15,8.40060e-16,1.49161e+00,"[comment, classifications, activity type, acti..."
4,"Phosphoric acid, fertiliser grade, without wat...",1.41518e-11,2.32285e-11,5.77226e-13,7.09802e-14,5.40247e-14,3.11832e-13,3.21460e-13,4.99749e-15,8.01974e-16,9.57087e-01,"[comment, classifications, activity type, acti..."
...,...,...,...,...,...,...,...,...,...,...,...,...
942,"Polyvinylchloride, suspension polymerised {GLO...",4.00758e-12,5.09850e-11,4.27138e-13,1.55799e-13,4.87316e-15,6.73882e-13,1.94120e-13,7.51997e-15,3.08085e-17,2.03593e+00,"[comment, classifications, activity type, acti..."
943,"Polyvinylidenchloride, granulate {RER}| market...",1.26105e-12,1.16702e-10,1.11385e-12,3.56608e-13,6.54782e-15,1.54516e-12,1.63779e-13,1.83718e-15,-2.86307e-18,5.33048e+00,"[comment, classifications, activity type, acti..."
944,"Polyvinylidenchloride, granulate {RoW}| market...",1.27545e-12,1.18372e-10,1.13303e-12,3.61710e-13,6.52482e-15,1.56688e-12,1.18197e-13,1.90118e-15,1.99466e-17,5.39975e+00,"[comment, classifications, activity type, acti..."
945,"Tetrafluoroethylene {GLO}| market for | APOS, U",2.03234e-11,1.90074e-10,2.87110e-11,5.80817e-13,9.83699e-15,6.18548e-12,2.57393e-12,3.20562e-11,1.58525e-15,1.37134e+02,"[comment, classifications, activity type, acti..."


In [137]:
df_sp_selected = df_sp_selected.iloc[[i for i in df_sp_selected.index if df_sp_selected.einame[i] is not None]]

In [138]:
# type(df_sp_selected.einame[665])

In [139]:
# type(activities_bw[0])

In [140]:
df_sp_selected.shape

(945, 12)

In [141]:
c = 0
for idx in df_sp_selected.index:
    if not isinstance(df_sp_selected.einame[idx], bw2data.backends.peewee.proxies.Activity):
        print(df_sp_selected.Activity[idx])
        print(idx)
        c += 1
c

0

In [142]:
dct_FUs = [{act: 1} for act in df_sp_selected.einame] # if isinstance(act, bw2data.backends.peewee.proxies.Activity)]
# dct_FUs = [{act: 1} for act in activities_bw]
# dct_FUs

In [143]:
# FU0 = dct_FUs[45]
# all_scores = {}
# lca = bw.LCA(FU0, pbs[0])
# lca.lci()
# lca.lcia()
# for category in pbs:
#     lca.switch_method(category)
#     lca.lcia()
#     all_scores[category] = {}
#     all_scores[category]["score"] = lca.score
#     all_scores[category]["unit"] = bw.Method(category).metadata["unit"]

In [144]:
# FU0

In [145]:
pd.options.display.float_format = '{:12.5e}'.format

In [146]:
# df_temp = pd.DataFrame.from_dict(all_scores).T
# df_temp

In [147]:
bw.calculation_setups

CalculationSetups dictionary with 1 object(s):
	chemicals

In [148]:
pbs + ["'ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq'"]

[('AESA (PBs-LCIA)', 'climate change', 'atmospheric CO2 concentration'),
 ('AESA (PBs-LCIA)', 'change in biosphere integrity', 'functional diversity'),
 ('AESA (PBs-LCIA)', 'ozone depletion', 'stratospheric O3 concentration'),
 ('AESA (PBs-LCIA)', 'freshwater use', 'global'),
 ('AESA (PBs-LCIA)', 'biogeochemical flows', 'phosphorus'),
 ('AESA (PBs-LCIA)', 'biogeochemical flows', 'nitrogen'),
 ('AESA (PBs-LCIA)',
  'climate change',
  'energy imbalance at top-of-atmosphere'),
 ('AESA (PBs-LCIA)', 'land-system change', 'global'),
 ('AESA (PBs-LCIA)', 'ocean acidification', 'carbonate ion concentration'),
 "'ReCiPe 2016 Midpoint (H) V1.03 / World (2010) H', 'Global warming', 'kg CO2 eq'"]

In [151]:
selected_methods = pbs + [
    m for k, v in rev_map_methods.items() for m in recipe2016 if str(k) in str(m)
]

In [152]:
bw.calculation_setups["chemicals"] = {
    "inv": dct_FUs,
    "ia": selected_methods,
}

In [153]:
# [act["name"] for FU in myMultiLCA.func_units for act in FU.keys()]

In [154]:
# bw.calculation_setups["chemicals"]
myMultiLCA = bw.MultiLCA("chemicals")
# myMultiLCA.results

df_impact = pd.DataFrame(
    data=myMultiLCA.results,
    columns=selected_methods,
    # index=[[act for FU in myMultiLCA.func_units for act in FU.keys()]]
    index=[
        tuple([act["name"], act["location"], act["reference product"]])
        for FU in myMultiLCA.func_units
        for act in FU
    ],
)
df_impact.head(3)

,"(AESA (PBs-LCIA), climate change, atmospheric CO2 concentration)","(AESA (PBs-LCIA), change in biosphere integrity, functional diversity)","(AESA (PBs-LCIA), ozone depletion, stratospheric O3 concentration)","(AESA (PBs-LCIA), freshwater use, global)","(AESA (PBs-LCIA), biogeochemical flows, phosphorus)","(AESA (PBs-LCIA), biogeochemical flows, nitrogen)","(AESA (PBs-LCIA), climate change, energy imbalance at top-of-atmosphere)","(AESA (PBs-LCIA), land-system change, global)","(AESA (PBs-LCIA), ocean acidification, carbonate ion concentration)","(ReCiPe 2016, 1.1 (20180117), Midpoint, Global Warming, 100 year timescale, Hierarchist)"
"(market for sulfuric acid, RoW, sulfuric acid)",3.68707e-12,2.24611e-14,1.15609e-15,1.53991e-12,2.04767e-16,4.67079e-14,4.97176e-14,1.09935e-16,1.12667e-14,1.60736e-01
"(market for sulfuric acid, RER, sulfuric acid)",2.68528e-12,1.86858e-14,8.42535e-16,1.08627e-12,2.16282e-16,4.97176e-14,3.62586e-14,1.23376e-16,8.20548e-15,1.23239e-01
"(market for sulfamic acid, GLO, sulfamic acid)",5.44322e-11,1.48524e-13,8.91329e-15,8.02670e-12,1.12230e-15,1.29056e-12,7.27725e-13,5.99318e-16,1.66331e-13,2.20238e+00


In [155]:
df_impact.shape

(945, 10)

In [156]:
# [(act["name"], act["location"]) for FU in myMultiLCA.func_units for act in FU]

In [157]:
indices = [tuple([act["name"], act["location"], act["reference product"]]) for act in df_sp_selected.einame]

In [158]:
# df_sp_selected

In [159]:
df_sp_selected["einame_idx"] = indices

C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_27228/218076019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp_selected["einame_idx"] = indices


In [160]:
df_sp_selected.set_index("einame_idx", inplace=True)
df_sp_selected.drop("Activity", 1, inplace=True)
df_sp_selected.drop("einame", 1, inplace=True)

C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_27228/3607502913.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sp_selected.drop("Activity", 1, inplace=True)
C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_27228/3607502913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sp_selected.drop("Activity", 1, inplace=True)
C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_27228/3607502913.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_sp_selected.drop("einame", 1, inplace=True)
C:\Users\ViteksPC\AppData\Local\Temp/ipykernel_27228/3607502913.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [161]:
df_sp_selected.shape

(945, 10)

In [162]:
import numpy as np

In [218]:
# np.isclose(7.40056E-13*10e13,1.05827e-13*10e13)

False

In [163]:
df_impact.shape

(945, 10)

In [209]:
axt = [a for a in eidb if 'sodium formate' in a["name"] and 'RoW' == a["location"]]# , 'green manure, Swiss integrated production, until March')
axt

['market for sodium formate' (kilogram, RoW, None)]

In [210]:
pbs[1]

('AESA (PBs-LCIA)', 'change in biosphere integrity', 'functional diversity')

In [223]:
lca = bw.LCA({axt[0]: 1}, pbs[1])
lca.lci()
lca.lcia()
lca.score

-1.181032442808871e-11

In [227]:
(-1.181032442808871e-11--7.17515E-12)/-7.17515E-12*100

64.60038365872087

In [205]:
res = (df_impact-df_sp_selected).div(df_sp_selected).mul(100)

In [206]:
# res

In [221]:
def make_pretty(styler):
    styler.background_gradient(axis=None, vmin=-100, vmax=100, cmap="seismic")
    return styler

In [222]:
res.style.pipe(make_pretty)